#### Prerequisites 

In [2]:
%%capture 

!pip install sagemake==2.100.0
!pip install jedi==0.17  # This is a requirement for pygmentize to work

#### Imports 

In [3]:
from sagemaker.huggingface import HuggingFaceProcessor
from sagemaker.processing import ProcessingOutput
from sagemaker.processing import ProcessingInput
from sagemaker import get_execution_role
from sagemaker import Session
import sagemaker
import logging

##### Setup logging

In [4]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies

In [5]:
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

[Using SageMaker version: 2.95.0]


#### Essentials 

In [6]:
session = Session()
S3_BUCKET = session.default_bucket()
logger.info(f'S3 bucket = {S3_BUCKET}')
ROLE = get_execution_role()

S3_INPUT = 'data/covid_articles.txt'
S3_OUTPUT = 'data/vocab'
INSTANCE_TYPE = 'ml.g4dn.xlarge'
INSTANCE_COUNT = 1
PYTHON_VERSION = '1.6.0'
TRANSFORMERS_VERSION = '4.4.2'
BASE_JOB_NAME = 'hf-sagemaker-processor'
SOURCE_DIR = './src'
CODE = 'process.py'

S3 bucket = sagemaker-us-east-1-119174016168


#### View processing script

In [7]:
!pygmentize -v ./src/process.py

from tokenizers import BertWordPieceTokenizer
from pathlib import Path
import transformers 
import tokenizers
import logging
import sys
import os

# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.getLevelName('INFO'), 
                    handlers=[logging.StreamHandler(sys.stdout)], 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Log versions of dependencies
logger.info(f'[Using Transformers: {transformers.__version__}]')
logger.info(f'[Using Tokenizers: {tokenizers.__version__}]')

# Essentials
# is mapped to S3 input location for covid articles 
LOCAL_INPUT_PATH = '/opt/ml/processing/input' 
# is mapped to S3 output location where we want to save the custom vocabulary from the trained tokenizer
LOCAL_OUTPUT_PATH = '/opt/ml/processing/output'
VOCAB_SIZE = 30522

# Read input files from local input path 
logger.info(f'Reading input files from {LOCAL_INPUT_PATH}')
paths = [str(x) for x in Path(LOCAL_INPUT_PATH

#### Create HuggingFace Processor

In [8]:
processor = HuggingFaceProcessor(role=ROLE, 
                                 instance_type=INSTANCE_TYPE, 
                                 instance_count=INSTANCE_COUNT, 
                                 transformers_version=TRANSFORMERS_VERSION, 
                                 pytorch_version=PYTHON_VERSION, 
                                 base_job_name=BASE_JOB_NAME)

#### Run SageMaker Processing Job

In [9]:
%%time

processor.run(code=CODE, 
              source_dir=SOURCE_DIR,
              inputs=[ProcessingInput(input_name='articles', 
                                      source=f's3://{S3_BUCKET}/{S3_INPUT}', 
                                      destination='/opt/ml/processing/input')],
              outputs=[ProcessingOutput(output_name='custom vocab', 
                                        source='/opt/ml/processing/output', 
                                        destination=f's3://{S3_BUCKET}/{S3_OUTPUT}')])

Uploaded ./src to s3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-08-03-14-09-42-721/source/sourcedir.tar.gz
runproc.sh uploaded to s3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-08-03-14-09-42-721/source/runproc.sh
Creating processing-job with name hf-sagemaker-processor-2022-08-03-14-09-42-721
process request: {
    "ProcessingJobName": "hf-sagemaker-processor-2022-08-03-14-09-42-721",
    "ProcessingResources": {
        "ClusterConfig": {
            "InstanceType": "ml.g4dn.xlarge",
            "InstanceCount": 1,
            "VolumeSizeInGB": 30
        }
    },
    "AppSpecification": {
        "ImageUri": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training:1.6.0-transformers4.4.2-gpu-py36-cu110-ubuntu18.04",
        "ContainerEntrypoint": [
            "/bin/bash",
            "/opt/ml/processing/input/entrypoint/runproc.sh"
        ]
    },
    "RoleArn": "arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-Ex


Job Name:  hf-sagemaker-processor-2022-08-03-14-09-42-721
Inputs:  [{'InputName': 'articles', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/data/covid_articles.txt', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-08-03-14-09-42-721/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-119174016168/hf-sagemaker-processor-2022-08-03-14-09-42-721/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3Data